Bias Mitigation using Reweighting 

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Example synthetic dataset
# X: Features, y: Labels, s: Sensitive attribute (e.g., gender, race)
np.random.seed(42)
X = np.random.rand(1000, 5)  # 1000 samples, 5 features
y = np.random.randint(0, 2, 1000)  # Binary target labels
s = np.random.randint(0, 2, 1000)  # Sensitive attribute (e.g., gender 0 or 1)

# Train-test split
X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(X, y, s, test_size=0.3, random_state=42)

# Step 1: Calculate reweighting factors
def calculate_weights(sensitive_attribute, labels):
    # Count frequency of each combination of sensitive attribute and label
    unique, counts = np.unique(np.stack((sensitive_attribute, labels), axis=1), axis=0, return_counts=True)
    
    # Calculate total number of instances per label and per sensitive attribute
    label_counts = np.bincount(labels)
    sensitive_counts = np.bincount(sensitive_attribute)
    
    # Compute weights for each group
    weights = np.zeros_like(labels, dtype=float)
    
    for (s_val, y_val), count in zip(unique, counts):
        # Weight = total instances / (instances in this group)
        weights[(sensitive_attribute == s_val) & (labels == y_val)] = (len(labels) / count)
    
    return weights

# Step 2: Apply bias mitigation by adjusting sample weights
weights = calculate_weights(s_train, y_train)

# Step 3: Train model using sample weights
model = LogisticRegression()
model.fit(X_train, y_train, sample_weight=weights)

# Step 4: Evaluate the model on the test set
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")

# Bias mitigation: Test for fairness
def demographic_parity_difference(predictions, sensitive_attribute):
    # Calculate demographic parity: P(y_pred = 1 | s = 0) - P(y_pred = 1 | s = 1)
    p_y_pred_1_s0 = np.mean(predictions[sensitive_attribute == 0] == 1)
    p_y_pred_1_s1 = np.mean(predictions[sensitive_attribute == 1] == 1)
    return p_y_pred_1_s0 - p_y_pred_1_s1

dp_diff = demographic_parity_difference(y_pred, s_test)
print(f"Demographic Parity Difference: {dp_diff:.4f}")

1. Preprocessing:

In this stage, we modify the dataset to remove bias. We will use the Reweighing method from AIF360 to adjust the dataset distribution.

2. In-processing:

This involves using a bias-mitigating algorithm during training. We will use the Adversarial Debiasing technique here.

3. Post-processing:

After the model is trained, post-processing ensures fairness by adjusting the predictions. We will use the Equalized Odds method for this.

All this is performed in the code below 


The code will print the accuracy of the model before and after post-processing and the Statistical Parity Difference before and after bias mitigation.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.datasets import AdultDataset
from aif360.sklearn.metrics import statistical_parity_difference
from tensorflow.compat.v1 import disable_v2_behavior, Session

# Disable TensorFlow 2 behavior for AIF360 compatibility
disable_v2_behavior()

# Load dataset: Adult Income dataset from AIF360
dataset = AdultDataset()

# Split dataset into train and test
train, test = dataset.split([0.7], shuffle=True)

# Preprocessing: Reweighing to ensure fairness in the dataset
RW = Reweighing(unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
train_transf = RW.fit_transform(train)

# In-processing: Adversarial Debiasing model to mitigate bias during training
sess = Session()
model = AdversarialDebiasing(privileged_groups=[{'sex': 1}], unprivileged_groups=[{'sex': 0}], 
                             scope_name='debiasing_classifier', sess=sess)
model.fit(train_transf)

# Evaluate the model on test set
test_pred = model.predict(test)
accuracy = accuracy_score(test_pred.labels, test.labels)
print(f"Accuracy (In-processing): {accuracy:.4f}")

# Bias Detection: Check for statistical parity difference before post-processing
metric = BinaryLabelDatasetMetric(test, unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
spd_before = metric.statistical_parity_difference()
print(f"Statistical Parity Difference (before post-processing): {spd_before:.4f}")

# Post-processing: Equalized Odds post-processing to further reduce bias
eq_odds = EqOddsPostprocessing(privileged_groups=[{'sex': 1}], unprivileged_groups=[{'sex': 0}])
eq_odds.fit(test, test_pred)
test_pred_transf = eq_odds.predict(test_pred)

# Bias Detection after post-processing
metric_post = BinaryLabelDatasetMetric(test_pred_transf, unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
spd_after = metric_post.statistical_parity_difference()
print(f"Statistical Parity Difference (after post-processing): {spd_after:.4f}")

# Final accuracy after post-processing
accuracy_post = accuracy_score(test_pred_transf.labels, test.labels)
print(f"Accuracy (Post-processing): {accuracy_post:.4f}")

1. Pre processing 


This stage adjusts the dataset to reduce bias before training. We’ll use the Reweighing technique.

Bias detection method used: Statistical Parity Difference is calculated after reweighing to check if the dataset has less bias.

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import BinaryLabelDatasetMetric

# Load dataset
dataset = AdultDataset()

# Split dataset into train and test
train, test = dataset.split([0.7], shuffle=True)

# Preprocessing: Reweighing to ensure fairness
RW = Reweighing(unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
train_transf = RW.fit_transform(train)

# Bias Detection: Statistical Parity Difference after preprocessing
metric = BinaryLabelDatasetMetric(train_transf, unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
spd = metric.statistical_parity_difference()
print(f"Statistical Parity Difference after Reweighing: {spd:.4f}")

2. In processing 


This approach integrates bias mitigation into the model training process. We’ll use Adversarial Debiasing during training.

Bias detection method used: The Statistical Parity Difference and Accuracy are evaluated after training the model to check for bias and performance.

import numpy as np
from sklearn.metrics import accuracy_score
from aif360.datasets import AdultDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.metrics import BinaryLabelDatasetMetric
from tensorflow.compat.v1 import disable_v2_behavior, Session

# Disable TensorFlow 2 behavior for AIF360 compatibility
disable_v2_behavior()

# Load dataset
dataset = AdultDataset()

# Split dataset into train and test
train, test = dataset.split([0.7], shuffle=True)

# In-processing: Adversarial Debiasing model to mitigate bias during training
sess = Session()
model = AdversarialDebiasing(privileged_groups=[{'sex': 1}], unprivileged_groups=[{'sex': 0}], 
                             scope_name='debiasing_classifier', sess=sess)
model.fit(train)

# Evaluate the model on test set
test_pred = model.predict(test)
accuracy = accuracy_score(test_pred.labels, test.labels)
print(f"Accuracy after In-processing (Adversarial Debiasing): {accuracy:.4f}")

# Bias Detection: Statistical Parity Difference after in-processing
metric = BinaryLabelDatasetMetric(test_pred, unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
spd = metric.statistical_parity_difference()
print(f"Statistical Parity Difference after Adversarial Debiasing: {spd:.4f}")

3. Post processing 


This stage adjusts the predictions of a trained model to improve fairness after training. We’ll use the Equalized Odds method.

Bias detection method used: Statistical Parity Difference and Accuracy to check if bias was reduced post-processing and how it affects the model’s performance.


Equalized Odds is applied after the model makes predictions, adjusting the outputs to reduce bias and ensure fairness.


import numpy as np
from sklearn.metrics import accuracy_score
from aif360.datasets import AdultDataset
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import BinaryLabelDatasetMetric

# Load dataset
dataset = AdultDataset()

# Split dataset into train and test
train, test = dataset.split([0.7], shuffle=True)

# For simplicity, let's assume we have already trained a model
# and have predictions on the test set (here we just copy the labels for demonstration).
test_pred = test.copy()

# Post-processing: Equalized Odds post-processing to reduce bias
eq_odds = EqOddsPostprocessing(privileged_groups=[{'sex': 1}], unprivileged_groups=[{'sex': 0}])
eq_odds.fit(test, test_pred)
test_pred_transf = eq_odds.predict(test_pred)

# Bias Detection: Statistical Parity Difference after post-processing
metric = BinaryLabelDatasetMetric(test_pred_transf, unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
spd = metric.statistical_parity_difference()
print(f"Statistical Parity Difference after Equalized Odds Post-processing: {spd:.4f}")

# Evaluate the final accuracy after post-processing
accuracy_post = accuracy_score(test_pred_transf.labels, test.labels)
print(f"Accuracy after Post-processing (Equalized Odds): {accuracy_post:.4f}")